In [1]:
import ipywidgets as widgets
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
import io
import importlib
import base64

#Upload button
upload_button = widgets.FileUpload(
    accept='.csv',  # Acceptera bara .csv-filer
    multiple=False  # Tillåt bara en fil att laddas upp
)

#Run button
run_button = widgets.Button(description="Run")

#Plot space
plot_output = widgets.Output()

#Output
out = widgets.Output()

#Download link
download_link = widgets.HTML("")

#Text info
filter_value_label = widgets.Label('Exclude values below:')

#Inut value
filter_value = widgets.FloatText(
    value=0,  #Default
)


def process_and_plot_data(button):
    global result        
    with out:
        out.clear_output()
        #Read upload data
        uploaded_file = upload_button.value[0]
        filename = uploaded_file['name']
        content = uploaded_file['content']
        data = pd.read_csv(io.BytesIO(content), sep=',', index_col=0)
        data.index = pd.to_datetime(data.index)
        print('First 10 rows of raw data')
        print(data.head(10))
        print('---')
        data.index.names=['time']
        data.columns=['value']
        #Filter "bad" data
        data=data.dropna()
        data = data[data['value'] > filter_value.value]
        

        

        #Run script
        script_module = importlib.import_module('autoMA')
        importlib.reload(script_module)
        result = script_module.run(data)

        # Plot data
        with plot_output:
            plot_output.clear_output()
            fig = go.Figure()
            data=data.dropna()
            fig.add_trace(go.Scatter(x=data.index, y=data['value'], mode='lines', name='Raw Data'))
            #Plot only if result exist
            if len(result) != 0:
                result=result.dropna()
                fig.add_trace(go.Scatter(x=result.index, y=result['value'], mode='lines', name='Result Data'))
            fig.show()

        #Download link
        csv = result.to_csv().encode('utf-8')
        b64 = base64.b64encode(csv)
        payload = b64.decode()
        download_link.value = f'<a download="result.csv" href="data:text/csv;base64,{payload}" target="_blank"><font size="3"><b>Download Result.csv</b></font></a>'

#Run button execute
run_button.on_click(process_and_plot_data)



#Show widgets
display(upload_button, filter_value_label, filter_value, run_button, out, plot_output, download_link)


FileUpload(value=(), accept='.csv', description='Upload')

Label(value='Exclude values below:')

FloatText(value=0.0)

Button(description='Run', style=ButtonStyle())

Output()

Output()

HTML(value='')